## Transorming races data and storing it in silver layer for future transformations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
races_ddl_schema = """
                   raceId INTEGER ,
                   year INTEGER ,
                   round INTEGER ,
                   circuitId INTEGER ,
                   name STRING ,
                   date DATE ,
                   time STRING ,
                   url STRING 
                   """

In [0]:
races_df = spark.read.format('csv').option('header', True).schema(races_ddl_schema).load(f"{raw_folder_path}/races.csv")

In [0]:
races_final_df = races_df.select(col("raceId"),col("year"),col("round"),col("circuitId"),col("name"),col("date"),col("time"))\
.withColumnsRenamed({"raceId": "race_id","year": "race_year","circuitId": "circuit_id" })\
.withColumn("race_timestamp", to_timestamp(concat(col("date"),lit(" "), col("time")),"yyyy-MM-dd HH:mm:ss"))\
    .withColumn("ingestion_date", current_timestamp()).drop("date","time")

### partition races data before process layer

In [0]:
races_final_df.write.format('parquet').mode('overwrite').partitionBy('race_year').option('path',f"{silver_folder_path}/races").save()

In [0]:
df = spark.read.format('parquet').option('header', True).option('inferSchema', True).load(f"{silver_folder_path}/races").display()